In [1]:
# !pip install tensorflow==2.3.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2

     |████████████████████████████████| 165.2 MB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 28.8 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 15.1 MB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 6.0 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  At

In [3]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [9]:
class ShowerEnv(Env):
    def __init__(self):
        #Actions we can take: down, stay, up -- discrete means three different values
        self.action_space = Discrete(3)
        #Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        #Set start temp
        self.state = 38 + random.randint(-3,3)
        #Set shower lenngth (seconds)
        self.shower_length = 60
        
    def step(self, action): #runns everytime we take the next step
        
        #0-1=-1 temp
        #1-1 = 0
        #2-1=1 temp
        self.state += action-1
        
        self.shower_length -= 1
        
        #Calculate reward
        if self.state > 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
            
        #Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False
            
        #Apply temp noise
        self.state += random.randint(-1,1)
        #placeholder -- something that openai gym requires
        info={}
        
        return self.state, reward, done, info
            
    def render(self):
        #implement visualizationn things
        pass
    
    def reset(self):
        #Reset shower temp
        self.state = 38 + random.randint(-3,3)
        #Rest shower time
        self.shower_length=60
        return self.state

In [12]:
env = ShowerEnv()

In [20]:
env.action_space.sample()

0

In [25]:
env.observation_space.sample()

array([92.388504], dtype=float32)

In [28]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-28
Episode:2 Score:-60
Episode:3 Score:-42
Episode:4 Score:-50
Episode:5 Score:-26
Episode:6 Score:-26
Episode:7 Score:-46
Episode:8 Score:-58
Episode:9 Score:-58
Episode:10 Score:-60


# Creating a Deep Learning Model with Keras

In [30]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [35]:
states = env.observation_space.shape
actions = env.action_space.n

In [36]:
actions

3

In [41]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states)) #input_shapes means removed flattened layer and skipped to dense layer
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [48]:
del model

In [49]:
model = build_model(states, actions)

In [50]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# Building the Agent with Keras-RL

In [54]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [59]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=200, target_model_update=1e-2)
    return dqn

In [61]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 0.4934
166 episodes - episode_reward: 29.542 [0.000, 54.000] - loss: 11.919 - mae: 24.697 - mean_q: 37.871

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 0.5858
167 episodes - episode_reward: 35.162 [8.000, 54.000] - loss: 11.960 - mae: 24.392 - mean_q: 37.563

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: 0.5838
167 episodes - episode_reward: 35.054 [-60.000, 58.000] - loss: 12.047 - mae: 25.042 - mean_q: 38.571

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 42s 4ms/step - reward: 0.5676
166 episodes - episode_reward: 33.976 [-12.000, 56.000] - loss: 11.828 - mae: 24.433 - mean_q: 37.612

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 45s 5ms/step - reward: 0.5518
done, t

In [62]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 56.000, steps: 60
Episode 3: reward: 58.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 52.000, steps: 60
Episode 6: reward: 58.000, steps: 60
Episode 7: reward: 54.000, steps: 60
Episode 8: reward: 58.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 52.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 58.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 58.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 56.000, steps: 60
Episode 18: reward: 54.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 58.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 58.000, steps: 60
Episode 26: reward: 56.000, st